In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from pathlib import Path
from datetime import datetime, timedelta
from pprint import pprint
from os import environ

import polars as pl
import cv2 as cv

from eott_dataset import *

In [ ]:
# %env set EOTT_DATASET_PATH
# %env set EOTT_OUTPUT_PATH

output_dir = Path(environ["EOTT_OUTPUT_PATH"]).expanduser().resolve()
print("output directory:")
print(output_dir)

pdf = read_participant_characteristics()
pdf

In [ ]:
def get_filename(source: str, ext="parquet"):
    return f"{source}.{ext}"


def with_pid_column(df: pl.DataFrame, value: int):
    df = df.with_row_index(name="frame")
    return df.with_columns(pid=pl.lit(value, pl.UInt8))


def logs_dataframe(p: Participant):
    df = with_pid_column(p.user_interaction_logs, p.pid)
    return df.select("pid", "frame", "index", "study", "event", "epoch")


def tobii_dataframe(p: Participant):
    print(p.pid)
    df = with_pid_column(p.tobii_gaze_predictions, p.pid)
    return df


ps = [Participant.from_dict(**row) for row in pdf.iter_rows(named=True)]

with pl.StringCache():
    pl.concat([logs_dataframe(p) for p in ps]).write_parquet(output_dir / get_filename("logs"))

pl.concat([tobii_dataframe(p) for p in ps]).write_parquet(output_dir / get_filename("tobii"))